In [1]:
import os
from datetime import datetime,timedelta
import sys
from importlib import reload 
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from steps.data_ingestion import *
from steps.data_inspection import *
from steps.data_visualization import *


from feast import FeatureStore

import mlflow
from mlflow.models import infer_signature
from mlflow.sklearn import log_model, load_model

import bentoml
from bentoml import HTTPServer




C:\Users\mkrym\AppData\Local\Temp\ipykernel_25188\1119964694.py:24: BentoMLDeprecationWarning: `bentoml.HTTPServer` is moved to `bentoml.legacy.HTTPServer` and will be removed in a future version.
  from bentoml import HTTPServer
C:\Python312\Lib\importlib\__init__.py:90: DeprecationWarning: bentoml.server module is deprecated, use bentoml.serve() instead.
  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
# Data ingest 

file_path = "C:\\Users\\mkrym\\Downloads\\archive.zip"

Ingestor = DataIngestorFactory.get_data_ingestor(file_path)   
df = Ingestor.ingest(file_path)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
# Initialize inspector and execute
inspector = DataInspector()
results = inspector.execute_all_strategies(df)

# Generate and print report
report = inspector.generate_report("inspection_report.txt")
print(report)

2025-08-23 10:55:58,388 - INFO - Executing all inspection strategies
2025-08-23 10:55:58,389 - INFO - Performing data type inspection
2025-08-23 10:55:58,392 - INFO - Generating summary statistics
2025-08-23 10:55:58,425 - INFO - Analyzing missing values
2025-08-23 10:55:58,428 - INFO - Detecting outliers
2025-08-23 10:55:58,443 - INFO - Checking for duplicates
2025-08-23 10:55:58,450 - INFO - Analyzing data distributions
2025-08-23 10:55:58,470 - INFO - Generating inspection report
2025-08-23 10:55:58,477 - INFO - Report saved to inspection_report.txt


Data Inspection Report - 2025-08-23 10:55:58

DataType:
--------------------------------------------------
Data Types:
  price: int64
  area: int64
  bedrooms: int64
  bathrooms: int64
  stories: int64
  mainroad: object
  guestroom: object
  basement: object
  hotwaterheating: object
  airconditioning: object
  parking: int64
  prefarea: object
  furnishingstatus: object
Type Summary:
  object: 7
  int64: 6

SummaryStatistics:
--------------------------------------------------
Numerical Summary:
  count: {'price': 545.0, 'area': 545.0, 'bedrooms': 545.0, 'bathrooms': 545.0, 'stories': 545.0, 'parking': 545.0}
  mean: {'price': 4766729.247706422, 'area': 5150.54128440367, 'bedrooms': 2.9651376146788992, 'bathrooms': 1.2862385321100918, 'stories': 1.8055045871559634, 'parking': 0.6935779816513762}
  std: {'price': 1870439.6156573922, 'area': 2170.141022508803, 'bedrooms': 0.7380638605685743, 'bathrooms': 0.5024696160532146, 'stories': 0.8674924629255264, 'parking': 0.8615857504605412}
 

In [4]:
# Sample inspection results (in practice, these would come from data_inspection.py)
inspection_results = {
    'MissingValues': {
        'missing_counts': df.isnull().sum().to_dict(),
        'missing_percentage': (df.isnull().sum() / len(df) * 100).to_dict()
    },
    'CorrelationAnalysis': {
        'correlations': df.select_dtypes(include=['float64', 'int64']).corr().to_dict()
    }
}

# Initialize visualizer and execute
visualizer = DataVisualizer()
visualizer.execute_all_visualizations(df, inspection_results)

2025-08-23 10:55:58,500 - INFO - Executing all visualization strategies
2025-08-23 10:55:58,501 - INFO - Running visualization: MissingValuesVisualization
2025-08-23 10:55:58,501 - INFO - Running visualization: MissingValuesVisualization
2025-08-23 10:55:58,503 - INFO - Visualizing missing values
2025-08-23 10:55:58,758 - INFO - Missing values plot saved as 'missing_values.png'
2025-08-23 10:55:58,759 - INFO - Running visualization: NumericalDistributionVisualization
2025-08-23 10:55:58,760 - INFO - Visualizing data distributions
2025-08-23 10:55:58,997 - INFO - Distribution plot for price saved as 'distribution_price.png'
2025-08-23 10:55:59,191 - INFO - Distribution plot for area saved as 'distribution_area.png'
2025-08-23 10:55:59,534 - INFO - Distribution plot for bedrooms saved as 'distribution_bedrooms.png'
2025-08-23 10:55:59,764 - INFO - Distribution plot for bathrooms saved as 'distribution_bathrooms.png'
2025-08-23 10:55:59,927 - INFO - Distribution plot for stories saved as 